In [1]:
import pandas as pd
import numpy as np
import os
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential, load_model
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


import dill as pickle

In [2]:
os.getcwd()  # Get the current working directory (cwd)

'/Users/ryanmackie/Documents/DSI_coursework/Submissions/Projects/capstone-master'

In [3]:
os.chdir('/Users/ryanmackie/Documents/DSI_coursework/Submissions/Projects/capstone-master')

In [4]:
os.getcwd()

'/Users/ryanmackie/Documents/DSI_coursework/Submissions/Projects/capstone-master'

In [12]:
update_df = pd.read_csv('./datasets/fake_news/train.csv')

print(f'The dataframe has {update_df.shape[0]} rows and {update_df.shape[1]} columns.')
update_df.head(6)

The dataframe has 20800 rows and 5 columns.


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0


In [13]:
update_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [14]:
update_df = update_df.dropna().reset_index(drop = True)

In [15]:
update_df = update_df.drop(columns = ['title','author', 'id'])

In [16]:
update_df.columns = ['Story', 'Label']

In [17]:
update_df['Label'] = update_df['Label'].map({1:0, 0:1})

In [18]:
update_df['Label'].value_counts(normalize = True)

1    0.566639
0    0.433361
Name: Label, dtype: float64

In [19]:
X = update_df['Story']
y = update_df['Label']

In [20]:
def to_words(series):
    ps = PorterStemmer()
    corpus = []
    start = time.time()
    for i in range(0, len(series)):
        letters = re.sub('[^a-zA-Z]', ' ',series[i])
        words = letters.lower().split()
        meaningful_words = [ps.stem(w) for w in words if not w in stopwords.words('english')]
        final = ' '.join(meaningful_words)
        corpus.append(final)
    end = time.time()
    print(f'Time Elapsed: {round(end - start, 2)} seconds')
    return corpus

In [21]:
vocab_size = round(20_000*1.3)

In [22]:
onehot = [one_hot(val,vocab_size)for val in to_words(X)]

Time Elapsed: 2332.49 seconds


In [1]:
# maxlen = 0
# for i in range(0, len(update_df['Story'])):
#     number = len(update_df['Story'][i].split())
#     if number > maxlen:
#         maxlen = number

padded_onehot = pad_sequences(onehot, padding = 'pre', maxlen = 2935)

In [27]:
X_final = np.array(padded_onehot)
y_final = np.array(y)

print(X_final.shape)
print(y_final.shape)

# X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = .2, random_state = 33)

(18285, 2935)
(18285,)


In [28]:
model = load_model('lstm_model.h5')

In [29]:
model.fit(X_final, y_final, epochs=2)

Epoch 1/2
572/572 [==============================] - 1332s 2s/step - loss: 0.3281 - acc: 0.8708
Epoch 2/2
572/572 [==============================] - 998s 2s/step - loss: 0.1423 - acc: 0.9573


In [30]:
model.save('lstm_model_2.h5')

In [51]:
# def model_func(layer_three_neurons=32, layer_four_neurons=16, layer_one_dropout=0.5, layer_two_dropout=0.5, layer_three_dropout=0.5, layer_four_dropout=0.5):
#     model = Sequential()
#     model.add(Embedding(vocab_size,output_dim=40,input_length=500))
#     model.add(Dropout(layer_one_dropout))
#     model.add(LSTM(256))
#     model.add(Dropout(layer_two_dropout))
#     model.add(Dense(layer_three_neurons, activation='relu'))
#     model.add(Dropout(layer_three_dropout))
#     model.add(Dense(layer_four_neurons, activation='relu'))
#     model.add(Dropout(layer_four_dropout))
#     model.add(Dense(1, activation='sigmoid')) # We can also use "linear"
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['acc'])
#     return model

# # define the model with the wrapper
# nn = KerasClassifier(build_fn=model_func, epochs=10, batch_size=512)

In [52]:
# # Params grid
# params = {
#     "epochs":[10, 20],
#     "layer_three_neurons":[20, 32],
#     "layer_four_neurons":[16, 32],
#     "layer_one_dropout":[0.1, 0.3, 0.5],
#     "layer_two_dropout":[0.1, 0.3, 0.5],
#     "layer_three_dropout":[0.1, 0.3, 0.5],
#     "layer_four_dropout":[0.1, 0.3, 0.5]
# }
# gs = GridSearchCV(estimator=nn, param_grid=params, cv=2) # I'm using cv=2 for the sake of time! 
# gs.fit(X_train, y_train)
# print(gs.best_score_)
# gs.best_params_